In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import json

with open('/content/drive/MyDrive/Colab Notebooks/NLP/layer1.json', 'r') as f:
    recipes = json.load(f)

In [4]:
recipes[1]

{'ingredients': [{'text': '1 c. elbow macaroni'},
  {'text': '1 c. cubed American cheese (4 ounce.)'},
  {'text': '1/2 c. sliced celery'},
  {'text': '1/2 c. minced green pepper'},
  {'text': '3 tbsp. minced pimento'},
  {'text': '1/2 c. mayonnaise or possibly salad dressing'},
  {'text': '1 tbsp. vinegar'},
  {'text': '3/4 teaspoon salt'},
  {'text': '1/2 teaspoon dry dill weed'}],
 'url': 'http://cookeatshare.com/recipes/dilly-macaroni-salad-49166',
 'partition': 'train',
 'title': 'Dilly Macaroni Salad Recipe',
 'id': '000033e39b',
 'instructions': [{'text': 'Cook macaroni according to package directions; drain well.'},
  {'text': 'Cold.'},
  {'text': 'Combine macaroni, cheese cubes, celery, green pepper and pimento.'},
  {'text': 'Blend together mayonnaise or possibly salad dressing, vinegar, salt and dill weed; add in to macaroni mix.'},
  {'text': 'Toss lightly.'},
  {'text': 'Cover and refrigeratewell.'},
  {'text': 'Serve salad in lettuce lined bowl if you like.'},
  {'text': '

In [5]:
recipes[0]['partition']


'train'

In [6]:
partitions = list(set([recipe['partition'] for recipe in recipes]))
print(partitions)


['val', 'train', 'test']


Divide the data into train, test and Validation dataset

In [7]:
train_recipes = [recipe for recipe in recipes if recipe['partition'] == 'train']
test_recipes = [recipe for recipe in recipes if recipe['partition'] == 'test']
val_recipes = [recipe for recipe in recipes if recipe['partition'] == 'val']

print(f"Number of recipes in train set: {len(train_recipes)}")
print(f"Number of recipes in test set: {len(test_recipes)}")
print(f"Number of recipes in validation set: {len(val_recipes)}")


Number of recipes in train set: 720639
Number of recipes in test set: 154045
Number of recipes in validation set: 155036


In [8]:
# Save the test_recipes variable to your Google Drive
with open('/content/drive/MyDrive/Colab Notebooks/NLP/test_recipes.json', 'w') as f:
    json.dump(test_recipes, f)

In [20]:
test_recipes[0]

{'ingredients': [{'text': '2 12 cups milk'},
  {'text': '1 12 cups water'},
  {'text': '14 cup butter'},
  {'text': 'mashed potatoes, 1 box, homestyle'},
  {'text': '1 (8 ounce) can whole kernel corn (drained)'},
  {'text': '1 cup cheddar cheese'},
  {'text': '1 cup French-fried onions'}],
 'url': 'http://www.food.com/recipe/crunchy-onion-potato-bake-479149',
 'partition': 'test',
 'title': 'Crunchy Onion Potato Bake',
 'id': '00003a70b1',
 'instructions': [{'text': 'Preheat oven to 350 degrees Fahrenheit.'},
  {'text': 'Spray pan with non stick cooking spray.'},
  {'text': 'Heat milk, water and butter to boiling; stir in contents of both pouches of potatoes; let stand one minute.'},
  {'text': 'Stir in corn.'},
  {'text': 'Spoon half the potato mixture in pan.'},
  {'text': 'Sprinkle half each of cheese and onions; top with remaining potatoes.'},
  {'text': 'Sprinkle with remaining cheese and onions.'},
  {'text': 'Bake 10 to 15 minutes until cheese is melted.'},
  {'text': 'Enjoy !'}

In [ ]:
import locale

locale.setlocale(locale.LC_ALL, 'en_US.utf8')

'en_US.utf8'

In [ ]:
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding
print(locale.getpreferredencoding())

UTF-8


In [6]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 73.9 MB/s eta 0:00:00


In [11]:
import transformers
from transformers import BertTokenizer
import random

# Initialize the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


# Define the number of samples to use for training
num_samples = 10000

# Take a random subset of the dataset
random.seed(42)
train_data_subset = random.sample(train_recipes, num_samples)

# Tokenize the recipe titles, ingredients, and instructions for the train set
train_tokens = []
for recipe in train_data_subset:
    title = recipe['title']
    ingredients = [ingr['text'] for ingr in recipe['ingredients']]
    instructions = [instr['text'] for instr in recipe['instructions']]
    recipe_text = title + ' ' + ' '.join(ingredients) + ' ' + ' '.join(instructions)
    
    encoded_inputs = tokenizer.encode_plus(
                        recipe_text,
                        add_special_tokens=True,
                        max_length=512,
                        padding='max_length',
                        return_attention_mask=True,
                        truncation= True,
                        return_tensors='pt'
                        )
    input_ids = encoded_inputs['input_ids']
    attention_mask = encoded_inputs['attention_mask']
    
    train_tokens.append((input_ids, attention_mask))

In [12]:
# Tokenize the recipe titles, ingredients, and instructions for the test set


# Take a random subset of the dataset
random.seed(42)
test_data_subset = random.sample(test_recipes, num_samples)
test_tokens = []
for recipe in test_data_subset:
    title = recipe['title']
    ingredients = [ingr['text'] for ingr in recipe['ingredients']]
    instructions = [instr['text'] for instr in recipe['instructions']]
    recipe_text = title + ' ' + ' '.join(ingredients) + ' ' + ' '.join(instructions)
    encoded_test = tokenizer.encode_plus(
                        recipe_text,
                        add_special_tokens=True,
                        max_length=512,
                        padding='max_length',
                        return_attention_mask=True,
                        truncation= True,
                        return_tensors='pt'
                        )
    input_ids = encoded_test['input_ids']
    attention_mask = encoded_test['attention_mask']
    
    test_tokens.append((input_ids, attention_mask))

# Tokenize the recipe titles, ingredients, and instructions for the validation set

# Take a random subset of the dataset
random.seed(42)
val_data_subset = random.sample(val_recipes, num_samples)
val_tokens = []
for recipe in val_data_subset:
    title = recipe['title']
    ingredients = [ingr['text'] for ingr in recipe['ingredients']]
    instructions = [instr['text'] for instr in recipe['instructions']]
    recipe_text = title + ' ' + ' '.join(ingredients) + ' ' + ' '.join(instructions)
    encoded_val = tokenizer.encode_plus(
                        recipe_text,
                        add_special_tokens=True,
                        max_length=512,
                        padding='max_length',
                        return_attention_mask=True,
                        truncation= True,
                        return_tensors='pt'
                        )
    input_ids = encoded_val['input_ids']
    attention_mask = encoded_val['attention_mask']
    
    val_tokens.append((input_ids, attention_mask))

In [13]:
# Save the tokenizer
tokenizer.save_pretrained("/content/drive/MyDrive/Colab Notebooks/NLP/tokenizer")

('/content/drive/MyDrive/Colab Notebooks/NLP/tokenizer/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/NLP/tokenizer/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/NLP/tokenizer/vocab.txt',
 '/content/drive/MyDrive/Colab Notebooks/NLP/tokenizer/added_tokens.json')

In [ ]:
import transformers
from transformers import BertForMaskedLM, BertTokenizer
import torch

# Load pre-trained BERT model
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

# Set the device to use
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

model= model.to(device)

# Prepare the model for training
model.train()

# Define the optimizer and learning rate scheduler
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)

# Define the checkpoint directory and frequency
checkpoint_dir = '/content/drive/MyDrive/Colab Notebooks/NLP/Trained_models'
checkpoint_freq = 10000

# Train the model
for epoch in range(3):
    print('Epoch:', epoch+1)
    total_loss = 0
    num_batches = 0
    num_correct = 0
    num_total = 0
    
    for i, tokens in enumerate(train_tokens):
        input_ids = tokens[0]
        attention_mask = tokens[1]
        
        # Truncate input if it exceeds the maximum sequence length
        max_len = 512
        if len(input_ids[0]) > max_len:
            input_ids = input_ids[:, :max_len]
            attention_mask = attention_mask[:, :max_len]
        
        # Create a copy of the input tokens and randomly mask some of them
        labels = input_ids.clone()
        masked_indices = torch.bernoulli(torch.full(labels.shape, 0.15)).bool()
        labels[~masked_indices] = -100  # Set the unmasked tokens to -100
        if (labels == input_ids).all():
            # no masked token, skip this batch
            continue

        # Compute the model's MLM loss
        outputs = model(input_ids=input_ids.to(device), attention_mask=attention_mask.to(device), labels=labels.to(device))
        loss = outputs.loss

        # Perform backpropagation and update the model's parameters
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        scheduler.step()
        
        total_loss += loss.item()
        num_batches += 1
        
        # Compute the accuracy on the training set
        preds = outputs.logits.argmax(dim=-1)
        num_correct += (preds == labels.to(device)).sum().item()
        num_total += labels.numel()
        
        # Save the model checkpoint periodically
        if i > 0 and i % checkpoint_freq == 0:
            checkpoint_path = checkpoint_dir + 'epoch' + str(epoch+1) + '_batch' + str(i) + '/'
            model.save_pretrained(checkpoint_path)
            tokenizer.save_pretrained(checkpoint_path)
            print('Model saved')
    
    # Compute the accuracy on the validation set
    num_val_correct = 0
    num_val_total = 0
    for tokens in val_tokens:
        input_ids = tokens[0]
        attention_mask = tokens[1]
        labels = input_ids.clone()
        masked_indices = torch.bernoulli(torch.full(labels.shape, 0.15)).bool()
        labels[~masked_indices] = -100
        if (labels == input_ids).all():
            # no masked token, skip this batch
            continue
        outputs = model(input_ids=input_ids.to(device), attention_mask=attention_mask.to(device), labels=labels.to(device))
        preds = outputs.logits.argmax(dim=-1)
        num_val_correct += (preds == labels.to(device)).sum().item()
        num_val_total += labels.numel()
    
    val_acc = num_val_correct / num_val_total
    
    # Save the model checkpoint at the end of the epoch
    epoch_checkpoint_path = checkpoint_dir + 'epoch' + str(epoch+1) + '_final/'
    model.save_pretrained(epoch_checkpoint_path)
    tokenizer.save_pretrained(epoch_checkpoint_path)
    print('Model saved at the end of the epoch')
    
    train_acc = num_correct / num_total
    print('Average loss:', total_loss/num_batches, 'Train accuracy:', train_acc*100, 'Validation accuracy:', val_acc*100)



Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch: 1
Model saved at the end of the epoch
Average loss: 2.483972153588757 Train accuracy: 14.392871093750001 Validation accuracy: 14.370976562500001
Epoch: 2
Model saved at the end of the epoch
Average loss: 2.4781536060495304 Train accuracy: 14.39458984375 Validation accuracy: 14.415859375
Epoch: 3
Model saved at the end of the epoch
Average loss: 2.4816476642403753 Train accuracy: 14.37939453125 Validation accuracy: 14.3890234375


### This model was trained for 10 epochs and we achieved accuracy of about 70%, the model was saved as Trained_modelsepoch10_final. 

In [22]:
from transformers import BertTokenizer, BertForMaskedLM
import torch
from sklearn.metrics.pairwise import cosine_similarity

# Set the device to use
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load pre-trained BERT model for masked LM
model = BertForMaskedLM.from_pretrained('bert-base-uncased').to(device)

#Load the pre-trained model 
model= model.from_pretrained('/content/drive/MyDrive/Colab Notebooks/NLP/Trained_modelsepoch10_final')
model= model.to(device)


# Load fine-tuned BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('/content/drive/MyDrive/Colab Notebooks/NLP/Trained_modelsepoch10_final')

# Tokenize the user input
user_input = 'I want to make a cake'
encoded_user_input = tokenizer.encode_plus(user_input, 
                                            add_special_tokens=True, 
                                            max_length=512, 
                                            padding='max_length', 
                                            truncation= True,
                                            return_attention_mask=True, 
                                            return_tensors='pt').to(device)

# Get the BERT embeddings for the user input
with torch.no_grad():
    try:
        user_output = model(**encoded_user_input)
        user_embedding = torch.mean(user_output[0], dim=1)
    except Exception as e:
        print(f"Error getting user embedding: {e}")
        user_embedding = None

# Get the BERT embeddings for the recipe text
recipe_embeddings = []
for recipe in test_recipes[:1000]:
    title = recipe['title']
    ingredients = [ingr['text'] for ingr in recipe['ingredients']]
    instructions = [instr['text'] for instr in recipe['instructions']]
    recipe_text = title + ' ' + ' '.join(ingredients) + ' ' + ' '.join(instructions)

    encoded_inputs = tokenizer.encode_plus(recipe_text, 
                                            add_special_tokens=True, 
                                            max_length=512, 
                                            padding='max_length', 
                                            truncation= True,
                                            return_attention_mask=True, 
                                            return_tensors='pt').to(device)
    with torch.no_grad():
        try:
            output = model(**encoded_inputs)
            recipe_embedding = torch.mean(output[0], dim=1)
            recipe_embeddings.append(recipe_embedding)
        except Exception as e:
            print(f"Error getting embedding for recipe {title}: {e}")

if user_embedding is not None:
    # Compute the cosine similarity between the user input and each recipe
    similarities = cosine_similarity(user_embedding.cpu(), torch.cat(recipe_embeddings).cpu(), dense_output=True)

    # Get the index of the top most similar recipe
    top_index = similarities.argsort()[0][::-1][0]

    # Print the title, ingredients and instructions of the top most similar recipe
    recipe = test_recipes[top_index]
    title = recipe['title']
    ingredients = [ingr['text'] for ingr in recipe['ingredients']]
    instructions = [instr['text'] for instr in recipe['instructions']]
    print(f"Title: {title}")
    print("Ingredients:")
    print('\n'.join(ingredients))
    print("Instructions:")
    print('\n'.join(instructions))
else:
    print("Could not get user embedding")


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Title: Wacky Chocolate Cake Pops
Ingredients:
2- 1/4 cups All-purpose Flour
1- 1/2 cup White Sugar
1/2 teaspoons Salt
1- 1/2 teaspoon Baking Soda
13 cups Unsweetened Cocoa Powder
1/2 cups Vegetable Oil
1- 1/2 cup Water
1- 1/2 Tablespoon White Vinegar
13 cups Butter
4 cups Powdered Sugar
13 cups Milk
1 teaspoon Pure Vanilla Extract
1/2 cups Unsweetened Cocoa Powder
14 ounces, weight Dark Chocolate Candy Melts
30 whole Cookie Sticks
1 cup Shredded Coconut
1 cup Crushed Chocolate Cookies
15 whole Plastic Spiders
15 whole Gummy Worms
Instructions:
CAKE Preheat oven to 350 degrees F. In a medium mixing bowl, mix all the dry cake ingredients (flour through cocoa powder) together with a spoon.
Add all of the wet ingredients (oil, water and vinegar) and stir until well-combined.
Pour into a 9x13 pan which has been prepared with cooking spray.
Bake for 30 minutes or until a toothpick inserted into center comes out clean.
Remove from oven and cool.
FROSTING In a medium mixing bowl, beat butter u

In [29]:
import requests
import spacy

nlp = spacy.load("en_core_web_sm")

ingredient_names = []
# dictionary to store total nutrient values
total_nutrients = {}

important_nutrients = [
    "Protein",
    "Total lipid (fat)",
    "Carbohydrate, by difference",
    "Fiber, total dietary",
    "Sugars, total including NLEA",
    "Calcium, Ca",
    "Iron, Fe",
    "Magnesium, Mg",
    "Phosphorus, P",
    "Potassium, K",
    "Sodium, Na",
    "Zinc, Zn",
    "Vitamin C, total ascorbic acid",
    "Thiamin",
    "Riboflavin",
    "Niacin",
    "Vitamin B-6",
    "Folate, total",
    "Vitamin B-12",
    "Vitamin A, RAE",
    "Vitamin E (alpha-tocopherol)",
    "Vitamin D (D2 + D3)",
    "Vitamin K (phylloquinone)"
]

for ingredient in ingredients:
    doc = nlp(ingredient)
    name = ""
    for token in doc:
        if token.pos_ == "NOUN" or token.pos_ == "PROPN":
            if name != "":
                name += " "
            name += token.text
    if "(" in name:
        name = name[:name.index("(")].strip()
    if "," in name:
        name = name[:name.index(",")].strip()
    if "to " in name:
        name = name[:name.index("to ")].strip()
    if name.endswith("s"):
        name = name[:-1]
    if "recipe" in name:
        name = name[:name.index("recipe")].strip()
    if "page" in name:
        name = name[:name.index("page")].strip()
    if "Note" in name:
        name = name[:name.index("Note")].strip()
    ingredient_names.append(name)
# print(ingredient_names)

# make request to USDA FoodData Central API for each ingredient and add the nutrient values to the total_nutrients dictionary
for ingredient in ingredient_names:
    response = requests.get("https://api.nal.usda.gov/fdc/v1/foods/search", params={"api_key": "wrOUlYRyTJUOBsGxpYlQPfOwIeukbnsVec7rrvGv", "query": ingredient})
    nutrients = response.json()["foods"][0]["foodNutrients"]
    for nutrient in nutrients:
        nutrient_name = nutrient['nutrientName']
        nutrient_number = float(nutrient['nutrientNumber'])
        unit_name = nutrient['unitName']
        if nutrient_name in total_nutrients:
            total_nutrients[nutrient_name] += nutrient_number
        else:
            total_nutrients[nutrient_name] = nutrient_number

# print the total nutrient values
print("\nNutrients in this Food Recipe are:\n")
nutrient_names = []
for nutrient_name, nutrient_number in sorted(total_nutrients.items()):
    if nutrient_name in important_nutrients:
        nutrient_names.append(nutrient_name)
#print((nutrient_names))
for nutrient in nutrient_names:
    print(f"\u2022 {nutrient}")



Nutrients in this Food Recipe are:

• Calcium, Ca
• Carbohydrate, by difference
• Fiber, total dietary
• Folate, total
• Iron, Fe
• Magnesium, Mg
• Niacin
• Phosphorus, P
• Potassium, K
• Protein
• Riboflavin
• Sodium, Na
• Sugars, total including NLEA
• Thiamin
• Total lipid (fat)
• Vitamin A, RAE
• Vitamin B-12
• Vitamin B-6
• Vitamin C, total ascorbic acid
• Vitamin D (D2 + D3)
• Vitamin E (alpha-tocopherol)
• Vitamin K (phylloquinone)
• Zinc, Zn


In [ ]:
from transformers import BertTokenizer, BertForMaskedLM
import torch
from sklearn.metrics.pairwise import cosine_similarity

# Set the device to use
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load pre-trained BERT model for masked LM
model = BertForMaskedLM.from_pretrained('bert-base-uncased').to(device)

# Load the saved model weights
model= model.from_pretrained('/content/drive/MyDrive/Colab Notebooks/NLP/epoch1_batch9000')
model= model.to(device)

# Load fine-tuned BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('/content/drive/MyDrive/Colab Notebooks/NLP/tokenizer')

# Tokenize the user input
user_input = 'I want to make a dish with a Icecream'
encoded_user_input = tokenizer.encode_plus(user_input, 
                                            add_special_tokens=True, 
                                            max_length=512, 
                                            padding='max_length', 
                                            truncation= True,
                                            return_attention_mask=True, 
                                            return_tensors='pt').to(device)

# Get the BERT embeddings for the user input
with torch.no_grad():
    try:
        user_output = model(**encoded_user_input)
        user_embedding = torch.mean(user_output[0], dim=1)
    except Exception as e:
        print(f"Error getting user embedding: {e}")
        user_embedding = None

# Get the BERT embeddings for the recipe text
recipe_embeddings = []
for recipe in test_recipes[:100]:
    title = recipe['title']
    ingredients = [ingr['text'] for ingr in recipe['ingredients']]
    instructions = [instr['text'] for instr in recipe['instructions']]
    recipe_text = title + ' ' + ' '.join(ingredients) + ' ' + ' '.join(instructions)

    encoded_inputs = tokenizer.encode_plus(recipe_text, 
                                            add_special_tokens=True, 
                                            max_length=512, 
                                            padding='max_length', 
                                            truncation= True,
                                            return_attention_mask=True, 
                                            return_tensors='pt').to(device)
    with torch.no_grad():
        try:
            output = model(**encoded_inputs)
            recipe_embedding = torch.mean(output[0], dim=1)
            recipe_embeddings.append(recipe_embedding)
        except Exception as e:
            print(f"Error getting embedding for recipe {title}: {e}")

if user_embedding is not None:
    # Compute the cosine similarity between the user input and each recipe
    similarities = cosine_similarity(user_embedding.cpu(), torch.cat(recipe_embeddings).cpu(), dense_output=True)

    # Get the index of the top most similar recipe
    top_index = similarities.argsort()[0][::-1][0]

    # Print the title, ingredients and instructions of the top most similar recipe
    recipe = test_recipes[top_index]
    title = recipe['title']
    ingredients = [ingr['text'] for ingr in recipe['ingredients']]
    instructions = [instr['text'] for instr in recipe['instructions']]
    print(f"Title: {title}")
    print("Ingredients:")
    print('\n'.join(ingredients))
    print("Instructions:")
    print('\n'.join(instructions))
else:
    print("Could not get user embedding")


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Title: Popcorn and Caramel Chocolate Candy Cones
Ingredients:
5 whole Ice Cream Cones (Regular Cake-Type Cones With A Flat Bottom)
4 ounces, weight Vanilla Candy Coating (4 Ounces Is 2 Squares Of Almond Bark)
Decorative Candy Sprinkles
1/2 cups Popcorn Kernels
1/4 teaspoons Salt
30 Caramel Candies (Pre-Wrapped Squares)
1 Tablespoon Water
1/2 cups Chocolate Chips
1/2 cups Heath English Toffee Bits
Instructions:
For the candy coated cones:
Lay a piece of parchment paper over a cookie sheet.
Add one of the candy coating squares into a ramekin or small bowl that has a diameter slightly larger than the top of the ice cream cone.
Melt the candy coating in a microwave for 30 second bursts making sure to stir between each 30 seconds.
Dip the top of two of the ice cream cones into the melted candy coating.
You may need to tilt the bowl slightly so the coating covers the area that is about an inch to two inches from the top of the cone.
Then sprinkle the decorative candies on top of the candy co

In [7]:
!pip install discord

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 42.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 13.8 MB/s eta 0:00:00


In [ ]:
!pip install -U discord.py
!pip install python-dotenv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
def generate_recipe(model, tokenizer, input_text, num_recipes=1):
    # Tokenize the input text
    encoded_input = tokenizer.encode_plus(
        input_text,
        add_special_tokens=True,
        max_length=512,
        padding='max_length',
        return_attention_mask=True,
        return_tensors='pt'
    )
    input_ids = encoded_input['input_ids']
    attention_mask = encoded_input['attention_mask']

    # Generate the recipe titles, ingredients, and instructions
    with torch.no_grad():
        outputs = model.generate(
            input_ids=input_ids.repeat(num_recipes, 1),
            attention_mask=attention_mask.repeat(num_recipes, 1),
            max_length=512,
            do_sample=True,
            num_return_sequences=num_recipes
        )

    # Decode the generated text
    generated_recipes = []
    for output in outputs:
        generated_recipe = tokenizer.decode(output, skip_special_tokens=True)
        generated_recipe = generated_recipe.split('INGREDIENTS')[0].strip()
        generated_recipe = generated_recipe.split('\n\n')[0].strip()
        generated_recipes.append(generated_recipe)

    # Return the top recipe title and instructions
    top_recipe_title = generated_recipes[0].split('\n')[0]
    top_recipe_instructions = '\n'.join(generated_recipes[0].split('\n\n')[1:])

    return top_recipe_title, top_recipe_instructions

In [8]:
!pip install stanfordnlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 4.0 MB/s eta 0:00:00


In [ ]:
!pip install discord.py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
!python '/content/drive/MyDrive/Colab Notebooks/NLP/'Bot.py

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
2023-05-10 17:36:52 INFO     discord.client logging in using static token
2023-05-10 17:36:53 INFO     discord.gateway Shard ID None has connected to Gateway (Session ID: 0b0d32331f5e36f79518e79bd0093dc5).
We have logged in as Recipe Recommendation Bot#1255
2023-05-10 17:38:25 WARNING  discord.gateway Shard ID None heartbeat blocked for more than 10 seconds.
Loop t